In [1]:
from pathlib import Path

import numpy as np
import shapely
import utils
from loguru import logger

In [2]:
data_dirpath = Path("../data")

district_json_filepath = data_dirpath / "cn_district.json"
district_csv_filepath = data_dirpath / "cn_district.csv"

city_json_filepath = data_dirpath / "cn_city.json"
city_csv_filepath = data_dirpath / "cn_city.csv"

province_json_filepath = data_dirpath / "cn_province.json"
province_csv_filepath = data_dirpath / "cn_province.csv"

border_json_filepath = data_dirpath / "cn_border.json"

In [3]:
district_data = utils.load_json(district_json_filepath)
district_df = utils.geojson_to_df(district_data)
district_df.to_csv(district_csv_filepath, index=False)

In [ ]:
features = []
district_polygons = np.array(utils.geojson_to_geometries(district_data))
for _, group in district_df.groupby(district_df["city_adcode"]):
    city_polygon = shapely.union_all(district_polygons[group.index])
    city_polygon = utils.orient_polygon(city_polygon)
    lon, lat = utils.get_geometry_center(city_polygon)
    geometry_dict = utils.geometry_to_geometry_dict(city_polygon)

    district_properties = group.iloc[0].to_dict()
    city_properties = {
        "province_name": district_properties["province_name"],
        "province_adcode": district_properties["province_adcode"],
        "city_name": district_properties["city_name"],
        "city_adcode": district_properties["city_adcode"],
        "short_name": utils.shorten_city_name(district_properties["city_name"]),
        "lon": round(lon, 6),
        "lat": round(lat, 6),
    }

    feature = utils.make_feature(geometry_dict, city_properties)
    features.append(feature)
    logger.info(city_properties)

city_data = utils.make_geojson(features)
utils.dump_json(city_json_filepath, city_data)

city_df = utils.geojson_to_df(city_data)
city_df.to_csv(city_csv_filepath, index=False)

In [ ]:
features = []
city_polygons = np.array(utils.geojson_to_geometries(city_data))
for _, group in city_df.groupby(city_df["province_adcode"]):
    province_polygon = shapely.union_all(city_polygons[group.index])
    province_polygon = utils.orient_polygon(province_polygon)
    lon, lat = utils.get_geometry_center(province_polygon)
    geometry_dict = utils.geometry_to_geometry_dict(province_polygon)

    city_properties = group.iloc[0].to_dict()
    province_properties = {
        "province_name": city_properties["province_name"],
        "province_adcode": city_properties["province_adcode"],
        "short_name": utils.shorten_province_name(city_properties["province_name"]),
        "lon": round(lon, 6),
        "lat": round(lat, 6),
    }

    feature = utils.make_feature(geometry_dict, province_properties)
    features.append(feature)
    logger.info(province_properties)

province_data = utils.make_geojson(features)
utils.dump_json(province_json_filepath, province_data)

province_df = utils.geojson_to_df(province_data)
province_df.to_csv(province_csv_filepath, index=False)

In [6]:
province_polygons = utils.geojson_to_geometries(province_data)
border_polygon = shapely.union_all(province_polygons)
border_polygon = utils.orient_polygon(border_polygon)
geometry_dict = utils.geometry_to_geometry_dict(border_polygon)
properties = {"country_name": "中华人民共和国", "country_adcode": 100000}
feature = utils.make_feature(geometry_dict, properties)
border_data = utils.make_geojson([feature])
utils.dump_json(border_json_filepath, border_data)